In [1]:
# David Simmons

In [2]:
from nltk import tokenize
import random
import pandas as pd
from datetime import datetime
import gensim

# Load Google's pre-trained Word2Vec model.
#model = gensim.models.Word2Vec.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)
word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

In [3]:
with open('female.txt', 'r', encoding='utf-8') as f:
    female_words = f.read().split()
    num_female_words = len(female_words)
    
female_words_w2v = []
female_words_not_w2v = []

for word in female_words:
    if word in word2vec.key_to_index.keys():
        female_words_w2v.append(word)
    else:
        female_words_not_w2v.append(word)
    
with open('male.txt', 'r', encoding='utf-8') as f:
    male_words = f.read().split()
    num_male_words = len(male_words)
    
male_words_w2v = []
male_words_not_w2v = []

for word in male_words:
    if word in word2vec.key_to_index.keys():
        male_words_w2v.append(word)
    else:
        male_words_not_w2v.append(word)

In [32]:
def obfuscate_text(row):
    text = row['post_text']
    parsed_text = tokenize.word_tokenize(text)
    obfuscated_text = ''
    if row['op_gender'] == 'W':
        for token in parsed_text:
            if token in female_words and random.random() <= 0.66667:
                obfuscated_text += male_words[random.randint(0, num_male_words-1)] + ' '
            else:
                obfuscated_text += token + ' '
        
    elif row['op_gender'] == 'M':
        for token in parsed_text:
            if token in male_words and random.random() <= 0.66667:
                obfuscated_text += female_words[random.randint(0, num_female_words-1)] + ' '
            else:
                obfuscated_text += token + ' '
        
    dataset.at[row.name, 'post_text'] = obfuscated_text

In [33]:
dataset = pd.read_csv('dataset.csv')

In [34]:
start_time = datetime.now()
dataset.apply(obfuscate_text, axis=1)
end_time = datetime.now()
total_time = end_time-start_time

print('total time:', total_time)

total time: 0:00:03.034435


In [35]:
dataset.to_csv('obfuscated_modified_dataset.csv', index=False)